In [3]:
import csv
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./data') if isfile(join('./data', f))]

In [10]:
import re
import json
from collections import defaultdict

In [5]:
selected_files =  ['realDonaldTrump', 'SecretaryCarson','MartinOMalley', 'JebBush',
             'BarackObama', 'HillaryClinton','BernieSanders']

In [9]:
def collect_data():
    text = list()
    labels = list()
    data_source = defaultdict(int)
    total_count = 12322
    not_count = 0
    for file in onlyfiles:        
        with open('data/' + file, newline='', encoding="utf8") as csv_file:
            if file[:-11] not in selected_files and 'trump' not in file.lower():
                continue                
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            if 'trump' not in file.lower():
                label = 'NoTrump'
                if not_count == total_count:
                    continue
            else:
                label = 'Trump'        
            text_ind = 1
            if "MORE_TRUMP.csv" in file:
                text_ind = 0
                source = 'realDonaldTrump'
            else:
                source = file[:-11]        
            for row in csv_reader:            
                if line_count != 0:
                    line = re.sub('[\s\n\t#@]+', ' ', row[text_ind])
                    line = re.sub(r'http\S+', '', line).strip()            
                    if len(line) == 0:
                        continue
                    text.append(line)                
                    labels.append(label)            
                    data_source[source] += 1
                    if label != 'Trump':
                        not_count += 1
                        if not_count == total_count:
                            break                
                line_count += 1
    from sklearn.model_selection import train_test_split
    train_data, test_data, train_labels, test_labels = \
    train_test_split(text, labels, test_size=0.13, random_state=42)
    with open("train.csv", "w", newline='', encoding='utf8') as csv_file:
        csv_writer = csv.writer(csv_file)
        for i in range(len(train_data)):
            csv_writer.writerow([train_data[i], train_labels[i]])
    with open("dev.csv", "w", newline='', encoding='utf8') as csv_file:
        csv_writer = csv.writer(csv_file)
        for i in range(len(test_data)):
            csv_writer.writerow([test_data[i], test_labels[i]])
                

## Word Difficulty clasification

In [83]:
import requests
import re
def get_word_difficulty(word):
    URL = "https://datayze.com/word-analyzer.php"      
    PARAMS = {'word':word}   
    # sending get request and saving the response as response object 
    r = requests.get(url = URL, params = PARAMS) 
    misspell_text = re.findall('<tr><td class="indented">Misspelled.*</tr>', str(r.content))
    misspell_value_text = re.findall('<span class="value">.* </span>', misspell_text[0])[0]   
    print(misspell_value_text)
    if 'Yes' in misspell_value_text:
        return None
    grade_text = re.findall('<td class="indented">Grade Level.*</td>', str(r.content))
    value_text = re.findall('<td><span class="value">.*</span></td></tr>', grade_text[0])[0]
    return value_text[24: value_text.index('</span>')]

In [124]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.stem import PorterStemmer
ps = PorterStemmer() 

In [158]:
primary_list = set()
sat_list = set()
toefl_list = set()
gre_list = set()
with open('primary_vocab.txt', 'r') as f:    
    for line in f:
        if line is not None:
            words = tokenizer.tokenize(line)
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            primary_list.update(words)
with open('SAT_vocab.txt', 'r') as f:
    for line in f:
         if line is not None:
            words = tokenizer.tokenize(line)
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            sat_list.add(words[0])
with open('TOEFL_vocab.txt', 'r') as f:
    for line in f:
         if line is not None:
            words = tokenizer.tokenize(line)
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            toefl_list.update(words)
with open('GRE_vocab.txt', 'r') as f:
    for line in f:
         if line is not None:
            words = tokenizer.tokenize(line) 
            if len(words) == 0:
                continue
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            gre_list.add(words[0])

In [159]:
words_level = dict()
for file in onlyfiles:       
    with open('data/' + file, newline='', encoding="utf8") as csv_file:        
        if file[:-11] not in selected_files and 'trump' not in file.lower():
            continue     
        csv_reader = csv.reader(csv_file, delimiter=',')                
        text_ind = 1
        if "MORE_TRUMP.csv" in file:
            text_ind = 0
            source = 'realDonaldTrump'
        else:
            source = file[:-11]
        if source not in words_level:
            words_level[source] = defaultdict(int)
        line_count = 0        
        for row in csv_reader:
            if line_count != 0:
                line = re.sub('[\s\n\t#@]+', ' ', row[text_ind])
                line = re.sub(r'http\S+', '', line).strip()            
                if len(line) == 0:
                    continue
                words = tokenizer.tokenize(row[text_ind])
                for w in words:
                    w = ps.stem(w)                    
                    value = 'others'
                    if w in gre_list:
                        value = 'GRE' 
                    elif w in toefl_list:
                        value = 'TOEFL' 
                    elif w in primary_list:
                        value = 'primary'                    
                    if value == 'others':
                        print(w)
                    words_level[source][value] += 1
                    words_level[source]['total'] += 1
            line_count += 1
                    
        

mattbevin
republican
kentucki
matt
republican
fred
keller
fought
pennsylvania
70
thousand
fred
john
brennan
mueller
drug
democrat
robert
mueller
dem
obama
amp
mueller
democrat
DO
shahira
knight
affair
levin
co
3f1xybks0r
johnni
co
exaaslgesh
NO
collus
NO
dem
40m
mueller
So
mcgahn
robert
mueller
trump
democrat
am
barrett
arizona
chairman
aerospac
flyfightwin
pennsylvania
republican
fred
keller
fred
militari
healthcar
mexico
mexico
amp
taxpay
w
mexico
am
mexico
democrat
amp
To
kentucki
matt
bevin
maga
whitehous
pennsylvania
co
wmeh9k9knv
megan
pennsylvania
megan
pennsylvania
co
v1e0d0ahho
pennsylvania
maga
co
6oqba5y5zu
montoursvil
pennsylvania
co
r67ixp3qpl
fred
keller
pennsylvania
republican
maga
montoursvil
pennsylvania
maga
co
ij2ajxfrza
iran
meantim
economi
iranian
statement
iran
On
cuba
freedom
amp
venezuelan
idli
cuba
berni
sander
sleepi
joe
biden
sleepi
joe
heap
sleepi
joe
democrat
hillari
phoni
russia
credibl
jerri
nadler
schiff
economi
seen
stuart
varney
foxandfriend
highli
els

trump
amp
foxnew
trump
bethlehem
pennsylvania
berni
sander
stuf
berni
foxnew
colorado
senmcsallyaz
arizona
No
collus
No
co
cbfoeeyltm
wisconsin
saturday
april
27th
resch
00pm
cdt
maga
co
bpyk8pf0o8
So
berni
foxnew
surprisingli
bretbaier
smiley
donnabrazil
No
collus
No
andi
mccarthi
column
devil
trump
russia
greatest
scam
mainstream
bigger
waterg
someday
god
forgotten
2016
steel
pace
hasn
seen
economi
So
notr
dame
pari
tanker
spoke
tigerwood
themast
amp
amp
comeback
importantli
medal
OF
freedom
minnesota
collus
cop
dem
hillari
mueller
democrat
trump
hater
clinton
focus
2016
russia
longer
legal
loophol
homeland
morgan
obama
chief
trump
cher
co
i5acsgurck
democrat
ON
MY
nanci
rep
omar
anti
semit
anti
israel
ungrat
statement
omar
nanci
mueller
mueller
No
collus
No
hillari
dnc
cop
boe
boe
737
max
featur
amp
rebrand
No
hell
telecom
huawei
obama
cyber
lobbyist
foxnew
stevehiltonx
stevehiltonx
trump
populist
trump
steve
puff
nanci
pelosi
60minut
amp
mueller
No
collus
wasn
comeback
co
41mtjtyej

watter
145
co
lotbt4rfjj
co
pywfgvguxx
witch
co
9w1iuge0d6
thousand
apprehend
captur
am
built
maga
wacki
nut
anncoult
hasn
entir
democrat
republican
sadli
unwil
am
built
california
highli
citizen
amp
california
enforc
apprehend
captur
built
greatli
repmiketurn
michael
cohen
pled
collus
loss
co
av9oi8xua
clue
co
xllpninobv
aluminum
aluminum
heather
wilson
finalist
texa
El
paso
septemb
2019
heather
fraudster
michael
cohen
sworn
NO
alabama
On
women
honor
women
co
vvnkubpmha
women
apprehend
strongest
economi
stuart
varney
foxandfriend
So
paul
manafort
loudli
NO
collus
russia
witch
statement
amp
amp
burr
So
cannot
dishonesti
foxandfriend
built
schedul
democrat
msnbc
cohen
cohen
seek
apprehend
democrat
violat
democrat
foxnew
democrat
feder
fiscal
prior
chri
stewart
No
democrat
hundr
subpoena
punctuat
workforc
polici
advisori
whitehous
co
izb2ttrinb
democrat
stronger
anti
semit
anti
semit
atroc
throughout
inconceiv
republican
constitution
amp
drug
traffick
amp
weirdo
steyer
gut
co
klmvscmene


color
god
mlkday
co
peavpcb8m4
manufactur
growth
1997
manufactur
wand
found
wand
No
donald
trump
bennett
stevehiltonx
schill
fame
marklevinshow
bob
kraft
belichick
bradi
entir
england
super
To
paid
amp
drug
crisi
WE
truli
co
wc9bytolkq
jame
co
itdceeiirv
co
ykcenjhzsk
korea
obama
rep
chairman
kim
co
zsmnftjedr
heard
gotten
nanci
am
shutdown
nanci
pelosi
irrat
amp
democrat
lefti
san
francisco
No
amnesti
daca
amnesti
bigger
000
000
nanci
nanci
pelosi
democrat
amp
drug
2020
economi
amp
Be
wouldn
heard
economi
economi
gdp
lowest
amp
mexico
caravan
mexico
mainstream
credibl
buzzfe
amp
economi
amp
heard
phoni
impeach
impeach
trump
newtgingrich
abraham
lincoln
unfairli
At
dover
truli
OF
buzzfe
dossier
paid
hillari
clinton
democrat
entir
russian
co
zi4012ld7
concern
crisi
shutdown
whitehous
seen
republican
No
spew
democrat
amp
drug
kevin
cork
foxnew
michael
cohen
perjuri
fraud
stolen
thousand
ly
caravan
democrat
jiminhof
So
nanci
pelosi
democrat
seven
800
000
paid
nanci
amp
wine
found
unreal
w

iran
150
terror
chuck
nanci
mcconnel
safer
taxpay
cannot
fbi
comey
fbi
chri
swecker
jame
comey
expos
stanc
fellow
democrat
2020
democrat
comey
fbi
vy
chief
wouldn
truli
washington
michael
anton
foxandfriend
built
democrat
militari
chuck
schumer
amp
nanci
pelosi
2006
democrat
longer
nanci
built
strictli
militari
easier
amp
built
amp
democrat
caravan
newli
built
makeshift
amp
fenc
amp
militari
mexico
fbi
jame
comey
acknowledg
knew
democrat
paid
phoni
trump
dossier
loudobb
jame
comey
collus
russia
fbi
witch
obama
cohen
witch
democrat
smock
trump
russia
jame
comey
No
smock
No
collus
foxnew
NO
collus
So
dem
wrongli
am
chief
certainti
nick
ayer
maga
trump
amp
truli
leakin
jame
comey
friday
untruth
rig
fraud
dishonest
expos
On
245
fbi
jame
comey
christoph
steel
honor
armynavygam
philadelphia
armywp_footbal
co
wdlkm6ve2t
armynavygam
co
yjhq6r6f9h
honor
co
byafesq8a
Da
nang
dick
blumenth
spew
braveri
vietnam
saw
As
bullet
whizz
Da
nang
dick
amp
pari
pari
agreement
lower
curv
collus
At
russian
c

...

makeamericagreatagain
trump2016
co
t1j0auwmxl
cnn
hater
oreillyfactor
greenskydeb
donaldtrump
ballot
trumptrain
trump2016
makeamericagreatagain
keksec__org
realdonaldtrump
polici
makeamericagreatagain
co
swxv3ycbqb
kdk144
realdonaldtrump
trump2016
makeamericagreatagainhttp
co
eufxeffu0d
newyorkvaluesin
buffalo
eve
nyprimari
GO
NY
votetrump
co
6x8dag7rea
1988
oprah
potu
trump2016
votetrumpni
primaryday
co
9rup33rl29
task
onurg
fiscal
co
q1nerotw9v
makeamericagreatagain
trump2016
co
awow5pyn7n
GO
backour
makeamericagreatagainwatch
amp
votetrump
co
lskdqgfyvq
icymi
trump2016
inbuffalo
votetrumpni
co
ts6mwn2oth
indianapoli
indiana
3pm
trump2016
makeamericagreatagainticket
co
36c7plaqyq
co
jbnytk3fy
eric
co
pu1vdiqlrv
am
foxandfriend
buffalo
nyprimari
co
z9qphvlzft
co
4ujduovlpj
makeamericagreatagain
nyprimaryhttp
co
awow5pyn7n
lyin
ted
cruz
superstorm
aid
septemb
11th
So
yorker
cruz
co
kyxbaw4fsl
kasich
hillari
threat
lyin
ted
cruz
am
hillari
amp
diamondandsilk
realdonaldtrump
rig
profit
a

711388380668493824
tucson
arizona
6000
votetrump
makeamericagreatagain
co
ctgyrlnawv
711366483872522240
andreatantaro
makeamericagreatagain
711353589394972674
jessebwatt
arizona
makeamericagreatagain
711350170798178304
diamondandsilk
wattersworld
trump2016
711349494701494272
arizona
20000
votetrump
makeamericagreatagain
co
pu5vb1pbo5
711275596626665472
cnn
75
trump
arizona
makeamericagreatagain
trump2016
co
uo287zgvvq
megynkelli
tank
megynkelli
saneplanet
utah
trumpcountri
utah
mormon
utahprimari
utah4trump
co
8e14re6yjo
teamtrumpaz
hanniti
trump
IN
phoenix
AZ
co
fj2j9lf2vh
eventbrit
donaldjtrumpjr
honor
utah
robert
oak
slc
711164130384674816
ma1973sk
foxnew
megynkelli
trump
ashleyedam
votetrump
711088013560778752
grammy620
1st
heard
trump
becam
closet
trump
veteran4trump
donald
trump
trump2016
wesleyrickard
sheriff
joe
arpaio
AZ
donald
trump
AZ
UT
co
4fzctzigq
cmichaeld2004
trump
70
percentil
gee
711084345356738560
ileanabarku
nytdavidbrook
nytdavidbrook
strictli
NO
makeamericagreatag

lightweight
florida
puppet
koch
alabama
So
am
republican
dem
indi
crossov
republican
hillari
carolina
repub
republican
establish
lightweight
marco
rubio
trump
koch
chosen
marco
rubio
lightweight
florida
charleshodgson1
bombshel
rubio
realdonaldtrump
co
tnenxt5ntdg
ilduce2016
100
realdonaldtrump
makeamericagreatagain
jimmythesaint09
realdonaldtrump
ronniememo
chrischristi
marcorubio
fairess369
marco
rubio
floridian
opportunist
phoni
sprivitor
realdonaldtrump
marco
amnesti
rubio
amnesti
co
d4zxazzbq8
alwaystrump
trump2016
trumptrain
703895967439970306
tiarardi
foxnew
realdonaldtrump
judgejeanin
GO
trump
ronniememo
donaldtrump
isn
chrischristi
stopcommoncor
makeamericagreatagain
co
xz1ecjpo8t
co
s8gq6cibhc
arkansa
amp
votetrump
amp
co
evoispn8px
erictrump
amp
stjude
memphi
TN
co
fuwhykhhnk
co
zgxukbnq3k
honor
marshal
israel
2004
co
g7g6hfp0dh
lightweight
marco
rubio
cruz
bigger
liar
rogerston
cnn
loss
loyal
built
fred
200
lightweight
rubio
democrat
carolina
republican
am
am
lobbyist
light

amp
co
tqej1eulvl
baton
roug
louisiana
WE
makeamericagreatagain
trump2016
co
xv7eie7a2l
baton
roug
louisiana
makeamericagreatagain
cruz
caught
denial
w
realbencarson
christian
co
p3ygl02aba
truthinvest
cnn
tedcruz
realdonaldtrump
ted
cruz
sleaz
realbencarson
baton
roug
louisiana
trump2016
makeamericagreatagain
cruz
sleazi
dishonest
pushpol
jeb
jeb
enlist
mommi
quicksand
iraq
120
republican
Hi
DT
markhalperin
morning_jo
jeb
40000000
hampshir
3000000
1st
worthless
nydailynew
mort
zuckerman
frantic
loser
clemson
carolina
makeamericagreatagain
scprimari
co
fgacmafxxc
haryd
realdonaldtrump
697595370940006400
jeb
697594507777482752
carolina
697568064611561472
So
jeb
highli
hampshir
carolina
worthless
nydailynew
politico
cred
worthless
nydailynew
dopey
mort
zuckerman
dopey
mort
zuckerman
worthless
nydailynew
markhalperin
hampshir
donaldtrump
berniesand
697286805125726208
hampshir
carolina
697263333959995393
hampshir
fitn
nhprimaryhttp
co
rbffcq8gqq
co
73aozk8umg
697263254083489792
drug
hampsh

9pm
est
trump2016
williebosshog
honor
forgotten
eric
pella
iowa
trump
pella
iowa
iacaucu
co
3kwol20zmm
So
makeamericagreatagain
sioux
iowa
truli
overflow
iowa
arappeport
nytim
co
fgvqsmbrkj
glennbeck
credibl
fox
clinton
mccain
co
dfcokb7ex8
megynkelli
sioux
counti
iowa
robert
jeffress
ted
cruz
citizen
cannot
am
iowa
phoni
anncoult
establish
mexico
greatest
trump2016
votetrump
co
tykxct5ztg
co
oc480lwvqg
noamscheib
dow
2000
pt
outperform
invest
jeb
jeb
690890625722396672
lukebrink
co
tcn6z3xwyu
am
iowa
thesouthwasrit
nro
subscript
idiot
realdonaldtrump
trump2016
rnc
truli
am
trump2016
co
6rh849dlyz
cruz
w
St
amp
goldman
sach
No
legal
disclosur
amp
sold
cruz
US
potu
minist
cruz
tarp
goldman
sach
disclos
puppet
glennbeck
brentbozel
lightweight
beg
jeb
mommi
jeb
mom
isi
putin
ted
cruz
reuter
goldman
sach
limbaugh
honor
trump2016
scam
washington
inaccur
dishonest
fox
trump2016
co
qknineus8h
zogbi
trump
cruz
rubio
nevada
iowa
gop
zogbi
makeamericagreatagain
trump2016http
co
ma6oabmypf
co
evq

In [160]:
words_level

{'realDonaldTrump': defaultdict(int,
             {'TOEFL': 124630,
              'total': 357788,
              'others': 63047,
              'primary': 153351,
              'GRE': 16760}),
 'SecretaryCarson': defaultdict(int,
             {'primary': 14905,
              'total': 37792,
              'TOEFL': 13765,
              'others': 7448,
              'GRE': 1674}),
 'MartinOMalley': defaultdict(int,
             {'primary': 19832,
              'total': 47548,
              'others': 11004,
              'TOEFL': 15060,
              'GRE': 1652}),
 'JebBush': defaultdict(int,
             {'primary': 23104,
              'total': 56243,
              'others': 13710,
              'TOEFL': 17147,
              'GRE': 2282}),
 'BarackObama': defaultdict(int,
             {'primary': 25305,
              'total': 58141,
              'TOEFL': 19206,
              'others': 11737,
              'GRE': 1893}),
 'HillaryClinton': defaultdict(int,
             {'primary': 20446

In [161]:
word_percent = dict()
for key, v_dict in words_level.items():    
    if key not in word_percent:
        word_percent[key] = defaultdict(float)
    for cate, count in v_dict.items():
        if cate == 'total':
            continue
        word_percent[key][cate] = count / words_level[key]['total']

In [162]:
word_percent

{'realDonaldTrump': defaultdict(float,
             {'TOEFL': 0.3483347680749494,
              'others': 0.176213288315986,
              'primary': 0.4286085614945163,
              'GRE': 0.04684338211454828}),
 'SecretaryCarson': defaultdict(float,
             {'primary': 0.39439563928873833,
              'TOEFL': 0.3642305249788315,
              'others': 0.19707874682472482,
              'GRE': 0.04429508890770534}),
 'MartinOMalley': defaultdict(float,
             {'primary': 0.4170943047026163,
              'others': 0.2314292925044166,
              'TOEFL': 0.31673256498696056,
              'GRE': 0.03474383780600656}),
 'JebBush': defaultdict(float,
             {'primary': 0.410788898173995,
              'others': 0.2437636683676191,
              'TOEFL': 0.3048734953683125,
              'GRE': 0.04057393809007343}),
 'BarackObama': defaultdict(float,
             {'primary': 0.43523503207719166,
              'TOEFL': 0.33033487556113583,
              'others': 

In [163]:
import json
with open('words_level.json', 'w') as fp:
    json.dump(word_percent, fp)